In [1]:
import numpy as np
import pandas as pd

class UniqueIdDataFrame(pd.DataFrame):
    def __init__(self, series : pd.Series ):
        super().__init__(data=series.values, columns=[series.name])

        name = series.name
        col_name_unique = '{}'.format(name)
        col_name_id = '{}_id'.format(name)
        unique = pd.unique(series.sort_values(ascending=True))
        self[col_name_id] = [np.where(unique == value)[0][0] for value in self[name]]

    #@property
    #def _constructor(self):
    #    return UniqueIdDataFrame

    def mergeTo(self, df):
        return df.merge(self)


df = pd.DataFrame([30, 10, 20, 40, 80, 60], columns=['ohs'])
df['linear'] = [i for i in range(len(df))]
df['ahs'] = [90, 40, 20, 50, 30, 40]

print('DataFrame:')
print(df)
print()

#[UniqueIdDataFrame(x).mergeTo(df, inplace=True) for x in [df['ohs'], df['ahs']]]
ohs_df = UniqueIdDataFrame(df['ohs'])
df = ohs_df.mergeTo(df)

ahs_df = UniqueIdDataFrame(df['ahs'])
df = ahs_df.mergeTo(df)

print('Merged DataFrame:')
print(df)
print()


DataFrame:
   ohs  linear  ahs
0   30       0   90
1   10       1   40
2   20       2   20
3   40       3   50
4   80       4   30
5   60       5   40

Merged DataFrame:
   ohs  linear  ahs  ohs_id  ahs_id
0   30       0   90       2       4
1   10       1   40       0       2
2   10       1   40       0       2
3   60       5   40       4       2
4   60       5   40       4       2
5   20       2   20       1       0
6   40       3   50       3       3
7   80       4   30       5       1



In [14]:
import numpy as np
import pandas as pd

class OneHotDataFrame(UniqueIdDataFrame):
    def __init__(self, series : pd.Series ):
        super().__init__(series)

        col_name_id = '{}_id'.format(series.name)
        unique = pd.unique(series.sort_values(ascending=True))
        fields = ['{}_{}'.format(series.name, i) for i in range(len(unique))]
        for index, value in enumerate(fields):
            self[value] = (self[col_name_id] == index).astype(int)

    #@property
    #def _constructor(self):
    #    return OneHotDataFrame

    def mergeTo(self, df):
        return df.merge(self)

df = pd.DataFrame([30, 10, 20, 40, 80, 60], columns=['ohs'])
df['linear'] = [i for i in range(len(df))]
df['ahs'] = [90, 40, 20, 50, 30, 40]
print('DataFrame:')
print(df)
print()

ohs_df = OneHotDataFrame(df['ohs'])
df = ohs_df.mergeTo(df)

ohs_df = OneHotDataFrame(df['ahs'])
df = ohs_df.mergeTo(df)

print('Merged DataFrame:')
print(df)
print()


DataFrame:
   ohs  linear  ahs
0   30       0   90
1   10       1   40
2   20       2   20
3   40       3   50
4   80       4   30
5   60       5   40

Merged DataFrame:
   ohs  linear  ahs  ohs_id  ohs_0  ohs_1  ohs_2  ohs_3  ohs_4  ohs_5  ahs_id  \
0   30       0   90       2      0      0      1      0      0      0       4   
1   10       1   40       0      1      0      0      0      0      0       2   
2   10       1   40       0      1      0      0      0      0      0       2   
3   60       5   40       4      0      0      0      0      1      0       2   
4   60       5   40       4      0      0      0      0      1      0       2   
5   20       2   20       1      0      1      0      0      0      0       0   
6   40       3   50       3      0      0      0      1      0      0       3   
7   80       4   30       5      0      0      0      0      0      1       1   

   ahs_0  ahs_1  ahs_2  ahs_3  ahs_4  
0      0      0      0      0      1  
1      0      0      1